En este archivo se explica e implementa la evaluación que los diferentes modelos dan a los conjuntos de test, y se calcula el rendimiento que estos modelos obtienen.

In [1]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_curve, roc_auc_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import f1_score, multilabel_confusion_matrix

### Predicción de clases
Para llevar a cabo la evaluación, un modelo entrenado debe asignar una clase a cada instancia del conjunto de test. Se implementa a continuación una función que, para cada versión ($set$) y partición ($fold$) del conjunto de datos, devuelve las predicciones de clase de un modelo ($model$) entrenado con el método seleccionado (method).

In [ ]:
def make_predictions(
    model: KNeighborsClassifier | SVC | GaussianNB | RandomForestClassifier,
    set: str, 
    fold: int,
    method_name: str
):
    # Cargamos los datos de test
    print(f"\nmodel: {model}, set: {set}, fold: {fold}")

    predictions_path = os.path.join(os.getcwd(), "predictions")
    data_path = os.path.join(os.getcwd(), "data")
    full_path = os.path.join(data_path, f"{set}/test{fold}_{set}.csv")

    test_df = pd.read_csv(full_path)

    X_test = test_df.iloc[:, :-1]
    y_test = test_df.iloc[:, -1]

    y_pred = model.predict(X_test)
    
    # Almacenamos las probabilidades de que cada instancia pertenezca a una clase
    y_scores = pd.DataFrame(model.predict_proba(X_test))
    y_scores.to_csv(os.path.join(predictions_path, f"pred_{fold}_{set}_{method_name}.csv"), index=False)
    

    # CURVA ROC
    # TODO: esto se debe cambiar porque pos_label indica la clase 
    # que se está considerando como positiva e y_score es en realidad
    # las probabilidades de que cada instancia pertenezca a dicha clase
    # ¿Se debe hacer considerando las 3 como positivas?

    # fpr, tpr, thresholds = roc_curve(y_test, y_score=y_pred, pos_label=2)
    
    
    # Área debajo de la curva
    roc_auc = roc_auc_score(y_test, y_scores, multi_class='ovr')
    
    cm = multilabel_confusion_matrix(y_test, y_pred)
    tn = cm[ : , 0, 0]
    tp = cm[ : , 1, 1]
    fn = cm[ : , 1, 0]
    fp = cm[ : , 0, 1]

    sensitivity = np.mean(tp / (tp + fn))
    #print("sensitivity=", sensitivity)

    accuracy = accuracy_score(y_test, y_pred)
    #print("accuracy=", accuracy)

    recall = sensitivity
    #print("recall=", recall)

    precision = np.mean(tp / (tp + fp))
    #print("precision=", precision)

    fnr = np.mean(fn / (fn + tp))
    #print("fnr=", fnr)

    fpr = np.mean(fp / (fp + tn))
    #print("fpr=", fpr)

    specificity = 1 - fpr
    # print("specificity=", specificity)

    f1 = 2 * precision * recall / (precision + recall)
    # print("f1=", f1)
    
    return [accuracy, sensitivity, specificity, precision, f1, fnr, fpr, roc_auc]


### Generación de métricas de modelos
A continuación, para cada versión y partición de los datos se va a generar las métricas de cada uno de los modelos generados anteriormente.

In [ ]:
METHODS = ['knn', 'svm', 'naive_bayes', 'random_forest']
SETS = [
    'norm', 'norm_PCA80', 'norm_PCA95',
    'original', 'original_PCA80', 'original_PCA95',
    'stand', 'stand_PCA80', 'stand_PCA95'
]

METRICS_NAMES = ['Accuracy', 'Sensitivity', 'Specificity', 'Precision', 'F1_Score', 'FNR', 'FPR', 'AUC']

for method in METHODS:
    for set in SETS:
        
        fold_metrics = []

        for fold in range(1, 6):

            # Cargamos el modelo
            model_name = f"{method}_{set}_{fold}.pkl"
            model_path = os.path.join(os.getcwd(), "models", model_name)
            model = joblib.load(model_path)

            # Calculamos su rendimiento
            metrics = make_predictions(model, set, fold, method)
            fold_metrics.append(metrics)

        # Creamos el DataFrame con los resultados de los folds
        df_metrics = pd.DataFrame(fold_metrics, columns=METRICS_NAMES)

        # Exportamos el CSV (creamos la carpeta si no existe)
        OUTPUT_DIR = 'metrics'
        
        try:
            os.mkdir(path=OUTPUT_DIR)
        except FileExistsError:
            pass

        file_name = f"{method}_{set}_metrics.csv"
        full_path = os.path.join(os.getcwd(), OUTPUT_DIR, file_name)
        df_metrics.to_csv(full_path, index=True, index_label="fold")
        
        print(f"Guardado: {file_name}")
            
            


model: KNeighborsClassifier(), set: norm, fold: 1

model: KNeighborsClassifier(), set: norm, fold: 2

model: KNeighborsClassifier(), set: norm, fold: 3

model: KNeighborsClassifier(), set: norm, fold: 4

model: KNeighborsClassifier(), set: norm, fold: 5
Guardado: knn_norm_metrics.csv

model: KNeighborsClassifier(), set: norm_PCA80, fold: 1

model: KNeighborsClassifier(), set: norm_PCA80, fold: 2

model: KNeighborsClassifier(), set: norm_PCA80, fold: 3

model: KNeighborsClassifier(), set: norm_PCA80, fold: 4

model: KNeighborsClassifier(), set: norm_PCA80, fold: 5
Guardado: knn_norm_PCA80_metrics.csv

model: KNeighborsClassifier(), set: norm_PCA95, fold: 1

model: KNeighborsClassifier(), set: norm_PCA95, fold: 2

model: KNeighborsClassifier(), set: norm_PCA95, fold: 3

model: KNeighborsClassifier(), set: norm_PCA95, fold: 4

model: KNeighborsClassifier(), set: norm_PCA95, fold: 5
Guardado: knn_norm_PCA95_metrics.csv

model: KNeighborsClassifier(), set: original, fold: 1

model: KNeighb